In [2]:
import csv
from itertools import islice, chain
from datetime import datetime
from collections import namedtuple, defaultdict, Counter

In [3]:
def cast_value(value):
    def my_cast():
        yield datetime, lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ")
        yield int, lambda x: int(x)
        yield str, lambda x: str(x)

    for expect_type, cast_fn in my_cast():
        try:
            return cast_fn(value)
        except ValueError:
            continue


def cast_data_row(data_row):
    return [cast_value(v) for v in data_row]

In [4]:
def read_file_rows(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        next(rows)  # skip headers
        for row in rows:
            yield cast_data_row(row)


def get_headers(file_name):
    with open(file_name) as f:
        rows = csv.reader(f, delimiter=',', quotechar='"')
        return next(rows)

In [5]:
def create_obj_from_file(file_name):
    obj = namedtuple(file_name.removesuffix('.csv').capitalize(), get_headers(file_name))
    data = read_file_rows(file_name)
    for row in data:
        yield obj(*row)

In [6]:
def create_obj_from_files(*file_names):
    headers = [get_headers(f) for f in file_names]
    column_names = headers[0] + [col for header in headers[1:] for col in header[1:]]
    CombinedRow = namedtuple('CombinedRow', column_names)
    for rows in zip(*(read_file_rows(f) for f in file_names)):
        combined_row = [rows[0][0]] + list(chain.from_iterable(row[1:] for row in rows))
        yield CombinedRow(*combined_row)

In [16]:
employment = create_obj_from_file('employment.csv')
pers_inf = create_obj_from_file('personal_info.csv')
upd_status = create_obj_from_file('update_status.csv')
vehicle = create_obj_from_file('vehicles.csv')

for i in islice(employment, 1):
    print(i)
for i in islice(pers_inf, 1):
    print(i)
for i in islice(upd_status, 1):
    print(i)
for i in islice(vehicle, 1):
    print(i)

Employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824')
Personal_info(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Update_status(ssn='100-53-9824', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
Vehicles(ssn='100-53-9824', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)


In [18]:
comb_obj = create_obj_from_files('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for i in islice(comb_obj, 3):
    print(i)

CombinedRow(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
CombinedRow(employer='Nicolas and Sons', department='Sales', employee_id='41-6841359', ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57), vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
CombinedRow(employer='Connelly Group', department='Research and Development', employee_id='98-7952860', ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), crea

In [23]:
def create_obj_non_stale_records(*file_names, cut_off_date=datetime(2017, 3, 1, 0, 0, 0)):
    headers = [get_headers(f) for f in file_names]
    column_names = headers[0] + [col for header in headers[1:] for col in header[1:]]
    CombinedRow = namedtuple('CombinedRow', column_names)

    last_updated_index = column_names.index('last_updated')

    for rows in zip(*(read_file_rows(f) for f in file_names)):
        combined_row = [rows[0][0]] + list(chain.from_iterable(row[1:] for row in rows))

        last_updated_dt = combined_row[last_updated_index]
        if last_updated_dt >= cut_off_date:
            yield CombinedRow(*combined_row)

In [25]:
comb_obj = create_obj_non_stale_records('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for i in islice(comb_obj, 1):
    print(i)

CombinedRow(employer='Stiedemann-Bailey', department='Research and Development', employee_id='29-0890771', ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30), vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)


In [19]:
def largest_car_make_by_gender(*file_names):
    make_counts_by_gender = defaultdict(Counter)
    for row in create_obj_from_files(*file_names):
        gender = row.gender
        vehicle_make = row.vehicle_make
        make_counts_by_gender[gender][vehicle_make] += 1

    largest_groups = {}
    for gender, make_counts in make_counts_by_gender.items():
        max_count = max(make_counts.values())
        largest_groups[gender] = [(make, count) for make, count in make_counts.items() if count == max_count]
    return largest_groups

In [20]:
largest_groups = largest_car_make_by_gender('employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv')
for gender, makes in largest_groups.items():
    print(f"Largest car make group(s) for {gender}:")
    for make, count in makes:
        print(f" - {make}: {count} vehicles")

Largest car make group(s) for Male:
 - Ford: 44 vehicles
Largest car make group(s) for Female:
 - Ford: 48 vehicles
 - Chevrolet: 48 vehicles
